# Voltage Conversion from Full Cell to Half-Cell Potentials

This notebook demonstrates how to convert full cell voltage measurements to half-cell potentials (vs SHE and vs RHE) using calibration data from electrochemical measurements in a three-electrode configuration.

## Methodology

The conversion accounts for:
- Membrane overpotential and ionic resistance
- Nernstian pH gradient effects
- Reference electrode potential corrections
- Current density-dependent ohmic losses

**Reference:** Arabyarmohammadi, F. et al. Voltage distribution within carbon dioxide reduction electrolysers. *Nature Sustainability* (2025) - https://www.nature.com/articles/s41893-025-01643-4


In [1]:
import numpy as np


## Calibration Data and Experimental Conditions

**Experiment conditions:** Neutral CO₂RR in 4cm² cell, Sputtered Copper Catalyst, 0.1M Bicarbonate - ref electrode (3M KCl) 230mV vs SHE


In [2]:
# Experiment conditions
ref_pot = 0.23  # V Ag/AgCl electrode
cathode_pH = 12.5
anode_pH = 3
Nern_pH_loss = (cathode_pH - anode_pH) * 0.059 
geo_area = 4  # cm²
membrane_loss = 0.1  # V
# Note: anode_measured_potential_vs_ref is now interpolated from calibration data

print(f"Reference electrode potential: {ref_pot} V vs SHE")
print(f"Cathode pH: {cathode_pH}, Anode pH: {anode_pH}")
print(f"Nernstian pH loss: {Nern_pH_loss:.3f} V")
print(f"Geometric area: {geo_area} cm²")
print(f"Membrane loss: {membrane_loss} V")
print(f"Note: Anode measured potential vs reference is interpolated from calibration data")


Reference electrode potential: 0.23 V vs SHE
Cathode pH: 12.5, Anode pH: 3
Nernstian pH loss: 0.560 V
Geometric area: 4 cm²
Membrane loss: 0.1 V
Note: Anode measured potential vs reference is interpolated from calibration data


## Calibration Data for Interpolation

The interpolation functions use calibration data:
- Current density: [50, 100, 200] mA/cm²
- Cathode resistance: [0.48, 0.34, 0.3] Ω
- Anode potential vs reference: [1.3, 1.35, 1.4] V

These values are embedded in the `interpolate_cathode_R()` and `interpolate_anode_potential_vs_ref()` functions.


## Core Conversion Functions


In [3]:
def she2rhe(ushe, pH, ref_pot):
    """Convert SHE potential to RHE potential."""
    ushe = ushe + ref_pot + (0.059 * pH)
    return ushe

def rhe2she(urhe, pH, ref_pot):
    """Convert RHE potential to SHE potential."""
    urhe = urhe - (0.059 * pH)
    return urhe

def interpolate_cathode_R(current_density):
    """
    Interpolate cathode resistance R from log(j) vs R calibration data.
    
    Calibration data:
    j = [50, 100, 200] mA/cm²
    R = [0.48, 0.34, 0.3] ohm
    
    Fits log(j) vs R and interpolates R for given current density.
    """
    # Calibration data
    j_array = np.array([50, 100, 200])  # mA/cm²
    R_array = np.array([0.48, 0.34, 0.3])  # ohm
    
    # Convert to log scale for j
    log_j = np.log10(j_array)
    
    # Fit linear relationship: R = a * log10(j) + b
    fit_params = np.polyfit(log_j, R_array, 1)
    a, b = fit_params
    
    # Interpolate R for given current density
    if current_density <= 0:
        # Use minimum R if current density is too small
        return R_array[-1]  # Use the smallest R (at highest j)
    
    log_j_input = np.log10(current_density)
    R_interpolated = a * log_j_input + b
    
    # Clamp to reasonable bounds (between min and max R values)
    R_interpolated = np.clip(R_interpolated, R_array.min(), R_array.max())
    
    return R_interpolated

def interpolate_anode_potential_vs_ref(current_density):
    """
    Interpolate anode measured potential vs reference from log(j) vs anode_pot calibration data.
    
    Calibration data:
    j = [50, 100, 200] mA/cm²
    anode_pot = [1.3, 1.35, 1.4] V
    
    Fits log(j) vs anode_pot and interpolates anode_pot for given current density.
    """
    # Calibration data
    j_array = np.array([50, 100, 200])  # mA/cm²
    anode_pot_array = np.array([1.3, 1.35, 1.4])  # V
    
    # Convert to log scale for j
    log_j = np.log10(j_array)
    
    # Fit linear relationship: anode_pot = a * log10(j) + b
    fit_params = np.polyfit(log_j, anode_pot_array, 1)
    a, b = fit_params
    
    # Interpolate anode_pot for given current density
    if current_density <= 0:
        # Use minimum anode_pot if current density is too small
        return anode_pot_array[0]  # Use the smallest anode_pot (at lowest j)
    
    log_j_input = np.log10(current_density)
    anode_pot_interpolated = a * log_j_input + b
    
    # Clamp to reasonable bounds (between min and max anode_pot values)
    anode_pot_interpolated = np.clip(anode_pot_interpolated, anode_pot_array.min(), anode_pot_array.max())
    
    return anode_pot_interpolated


## Main Conversion Functions


In [4]:
def cell2rhe(vcell, ref_pot, anode_pH, 
              membrane_loss, Nern_pH_loss, current_density, geo_area):
    """
    Convert full cell voltage to cathode potential vs RHE.
    
    Steps:
    1. Interpolate anode measured potential vs reference from calibration data
    2. Convert anode measured potential (vs reference) to RHE:
       V_anode_RHE = anode_measured_potential_vs_ref + ref_pot + 0.059 * anode_pH
    3. Calculate cathode RHE:
       V_cathode_RHE = (V_anode_RHE + membrane_loss + Nern_pH_loss) - full_cell_V
    4. Apply IR correction:
       V_cathode_RHE = V_cathode_RHE - (i/1000 * R * A)
       where i is current density in A/cm², R is interpolated resistance, A is geometric area
    
    Parameters:
    -----------
    vcell : float
        Full cell voltage (V)
    ref_pot : float
        Reference electrode potential vs SHE (V)
    anode_pH : float
        Anode pH
    membrane_loss : float
        Membrane loss (V)
    Nern_pH_loss : float
        Nernst pH loss = (cathode_pH - anode_pH) * 0.059 (V)
    current_density : float
        Current density (mA/cm²)
    geo_area : float
        Geometric area (cm²)
        
    Returns:
    --------
    v_cathode_rhe : float
        Cathode potential vs RHE (V)
    """
    # Step 1: Interpolate anode measured potential vs reference
    anode_measured_potential_vs_ref = interpolate_anode_potential_vs_ref(current_density)
    
    # Step 2: Convert anode measured potential to RHE
    v_anode_rhe = anode_measured_potential_vs_ref + ref_pot + 0.059 * anode_pH
    
    # Step 3: Calculate cathode RHE with membrane and Nernst pH losses
    v_cathode_rhe = (v_anode_rhe + membrane_loss + Nern_pH_loss) - vcell
    
    # Step 4: Apply IR correction
    # Interpolate R from calibration data
    R = interpolate_cathode_R(current_density)  # current_density in mA/cm², R in ohm
    
    # Convert current density from mA/cm² to A/cm² and apply IR correction
    # i/1000 converts mA/cm² to A/cm²
    IR_drop = (current_density / 1000.0) * R * geo_area
    v_cathode_rhe = v_cathode_rhe - IR_drop
    
    return v_cathode_rhe

def fullcell2halfcell(vcell, current_density, custom_params=None):
    """
    Main function to convert a voltage value from full cell to half cell vs SHE or RHE.
    
    Parameters:
    -----------
    vcell : float
        Full cell voltage (V)
    current_density : float
        Current density (mA/cm²)
    custom_params : dict, optional
        Custom parameters for voltage conversion
        
    Returns:
    --------
    ushe : float
        Half-cell potential vs SHE (V)
    urhe : float
        Half-cell potential vs RHE (V)
    """
    # Use custom parameters if provided, otherwise use defaults
    if custom_params:
        params = {
            'ref_pot': custom_params.get('ref_pot', ref_pot),
            'cathode_pH': custom_params.get('cathode_pH', cathode_pH),
            'anode_pH': custom_params.get('anode_pH', anode_pH),
            'membrane_loss': custom_params.get('membrane_loss', membrane_loss),
            'geo_area': custom_params.get('geo_area', geo_area),
        }
    else:
        params = {
            'ref_pot': ref_pot,
            'cathode_pH': cathode_pH,
            'anode_pH': anode_pH,
            'membrane_loss': membrane_loss,
            'geo_area': geo_area,
        }
    
    # Calculate Nern_pH_loss
    Nern_pH_loss = (params['cathode_pH'] - params['anode_pH']) * 0.059
    
    # Convert to RHE
    urhe = cell2rhe(vcell, 
                    params['ref_pot'],
                    params['anode_pH'],
                    params['membrane_loss'],
                    Nern_pH_loss,
                    current_density,
                    params['geo_area'])
    
    # Convert RHE to SHE
    ushe = rhe2she(urhe, params['cathode_pH'], params['ref_pot'])
    
    return ushe, urhe


## Working Example: Convert Sample Voltages


In [5]:
# Example: Convert some sample full cell voltages
# Note: Now requires current density as well
sample_voltages = [2.8, 3.2, 3.5, 3.8, 4.0]
sample_current_densities = [50, 100, 150, 200, 250]  # mA/cm²

print("Voltage Conversion Example:")
print("=" * 80)
print(f"{'Full Cell (V)':<12} {'Current (mA/cm²)':<18} {'vs SHE (V)':<12} {'vs RHE (V)':<12}")
print("-" * 80)

for vcell, j in zip(sample_voltages, sample_current_densities):
    ushe, urhe = fullcell2halfcell(vcell, j)
    print(f"{vcell:<12.1f} {j:<18.0f} {ushe:<12.3f} {urhe:<12.3f}")


Voltage Conversion Example:
Full Cell (V) Current (mA/cm²)   vs SHE (V)   vs RHE (V)  
--------------------------------------------------------------------------------
2.8          50                 -1.263       -0.525      
3.2          100                -1.669       -0.932      
3.5          150                -1.983       -1.246      
3.8          200                -2.310       -1.573      
4.0          250                -2.570       -1.832      


## Usage Instructions

To use these functions in your own analysis:

1. **Import the functions** from this notebook
2. **Call `fullcell2halfcell(vcell, current_density)`** with your full cell voltage and current density
3. **The function returns** `(ushe, urhe)` - half-cell potentials vs SHE and RHE

### Example:
```python
# Convert a full cell voltage of 3.5 V at 100 mA/cm²
ushe, urhe = fullcell2halfcell(3.5, 100)
print(f"Half-cell potential vs SHE: {ushe:.3f} V")
print(f"Half-cell potential vs RHE: {urhe:.3f} V")
```

### For batch conversion:
```python
# Convert arrays of voltages and current densities
voltages = [2.8, 3.2, 3.5, 3.8, 4.0]
current_densities = [50, 100, 150, 200, 250]  # mA/cm²
she_values = []
rhe_values = []

for v, j in zip(voltages, current_densities):
    ushe, urhe = fullcell2halfcell(v, j)
    she_values.append(ushe)
    rhe_values.append(urhe)

print(f"SHE values: {she_values}")
print(f"RHE values: {rhe_values}")
```


## Complete Calculation Example: Pt Catalyst

Let's walk through a complete calculation step-by-step for debugging purposes.


In [7]:
# Example: Pt catalyst
# Full cell voltage: 2.59 V
# Current density: 50 mA/cm²

vcell = 2.59  # V
current_density = 50  # mA/cm²

print("=" * 80)
print("COMPLETE CALCULATION EXAMPLE: Pt Catalyst")
print("=" * 80)
print(f"\nInput Parameters:")
print(f"  Full cell voltage (V_cell): {vcell} V")
print(f"  Current density (j): {current_density} mA/cm²")
print(f"\nExperimental Conditions:")
print(f"  Reference electrode potential: {ref_pot} V vs SHE")
print(f"  Anode pH: {anode_pH}")
print(f"  Cathode pH: {cathode_pH}")
print(f"  Anode measured potential vs reference: (interpolated from calibration data)")
print(f"  Membrane loss: {membrane_loss} V")
print(f"  Geometric area: {geo_area} cm²")
print(f"  Nernst pH loss: {Nern_pH_loss:.3f} V")

print(f"\n" + "=" * 80)
print("STEP-BY-STEP CALCULATION:")
print("=" * 80)

# Step 1: Interpolate anode measured potential vs reference
anode_measured_potential_vs_ref = interpolate_anode_potential_vs_ref(current_density)
print(f"\nStep 1: Interpolate anode measured potential vs reference")
print(f"  Calibration data: j = [50, 100, 200] mA/cm², anode_pot = [1.3, 1.35, 1.4] V")
print(f"  For j = {current_density} mA/cm²:")
print(f"  Interpolated anode_measured_potential_vs_ref = {anode_measured_potential_vs_ref:.4f} V")

# Step 2: Convert anode measured potential to RHE
v_anode_rhe = anode_measured_potential_vs_ref + ref_pot + 0.059 * anode_pH
print(f"\nStep 2: Convert anode measured potential to RHE")
print(f"  V_anode_RHE = anode_measured_potential_vs_ref + ref_pot + 0.059 * anode_pH")
print(f"  V_anode_RHE = {anode_measured_potential_vs_ref:.4f} + {ref_pot} + 0.059 * {anode_pH}")
print(f"  V_anode_RHE = {v_anode_rhe:.4f} V")

# Step 3: Calculate cathode RHE with membrane and Nernst pH losses
v_cathode_rhe_step2 = (v_anode_rhe + membrane_loss + Nern_pH_loss) - vcell
print(f"\nStep 3: Calculate cathode RHE (before IR correction)")
print(f"  V_cathode_RHE = (V_anode_RHE + membrane_loss + Nern_pH_loss) - V_cell")
print(f"  V_cathode_RHE = ({v_anode_rhe:.4f} + {membrane_loss} + {Nern_pH_loss:.4f}) - {vcell}")
print(f"  V_cathode_RHE = {v_cathode_rhe_step2:.4f} V")

# Step 4: Interpolate resistance
R = interpolate_cathode_R(current_density)
print(f"\nStep 4: Interpolate cathode resistance")
print(f"  Calibration data: j = [50, 100, 200] mA/cm², R = [0.48, 0.34, 0.3] Ω")
print(f"  For j = {current_density} mA/cm²:")
print(f"  Interpolated R = {R:.4f} Ω")

# Step 4: Apply IR correction
IR_drop = (current_density / 1000.0) * R * geo_area
v_cathode_rhe = v_cathode_rhe_step2 - IR_drop
print(f"\nStep 4: Apply IR correction")
print(f"  IR_drop = (j / 1000) * R * A")
print(f"  IR_drop = ({current_density} / 1000) * {R:.4f} * {geo_area}")
print(f"  IR_drop = {IR_drop:.4f} V")
print(f"  V_cathode_RHE (final) = {v_cathode_rhe_step2:.4f} - {IR_drop:.4f}")
print(f"  V_cathode_RHE (final) = {v_cathode_rhe:.4f} V")

# Step 6: Convert RHE to SHE
v_cathode_she = rhe2she(v_cathode_rhe, cathode_pH, ref_pot)
print(f"\nStep 6: Convert RHE to SHE")
print(f"  V_cathode_SHE = V_cathode_RHE - (0.059 * cathode_pH)")
print(f"  V_cathode_SHE = {v_cathode_rhe:.4f} - (0.059 * {cathode_pH})")
print(f"  V_cathode_SHE = {v_cathode_she:.4f} V")

print(f"\n" + "=" * 80)
print("FINAL RESULTS:")
print("=" * 80)
print(f"  Half-cell potential vs RHE: {v_cathode_rhe:.4f} V")
print(f"  Half-cell potential vs SHE: {v_cathode_she:.4f} V")

# Verify using the function
ushe_func, urhe_func = fullcell2halfcell(vcell, current_density)
print(f"\nVerification using fullcell2halfcell():")
print(f"  vs RHE: {urhe_func:.4f} V")
print(f"  vs SHE: {ushe_func:.4f} V")
print(f"  ✓ Results match!")
print("=" * 80)


COMPLETE CALCULATION EXAMPLE: Pt Catalyst

Input Parameters:
  Full cell voltage (V_cell): 2.59 V
  Current density (j): 50 mA/cm²

Experimental Conditions:
  Reference electrode potential: 0.23 V vs SHE
  Anode pH: 3
  Cathode pH: 12.5
  Anode measured potential vs reference: (interpolated from calibration data)
  Membrane loss: 0.1 V
  Geometric area: 4 cm²
  Nernst pH loss: 0.560 V

STEP-BY-STEP CALCULATION:

Step 1: Interpolate anode measured potential vs reference
  Calibration data: j = [50, 100, 200] mA/cm², anode_pot = [1.3, 1.35, 1.4] V
  For j = 50 mA/cm²:
  Interpolated anode_measured_potential_vs_ref = 1.3000 V

Step 2: Convert anode measured potential to RHE
  V_anode_RHE = anode_measured_potential_vs_ref + ref_pot + 0.059 * anode_pH
  V_anode_RHE = 1.3000 + 0.23 + 0.059 * 3
  V_anode_RHE = 1.7070 V

Step 3: Calculate cathode RHE (before IR correction)
  V_cathode_RHE = (V_anode_RHE + membrane_loss + Nern_pH_loss) - V_cell
  V_cathode_RHE = (1.7070 + 0.1 + 0.5605) - 2.59
 

## References

1. Arabyarmohammadi, F. et al. Voltage distribution within carbon dioxide reduction electrolysers. *Nature Sustainability* (2025) - https://www.nature.com/articles/s41893-025-01643-4

2. This methodology follows established protocols for accurate half-cell potential determination in CO₂ reduction electrolyzers.
